# FBCSP + LDA (intra-sujeto y cross-sujeto)

### Bloque 1 — Rutas de salida + utilidades de logging y guardado

Qué hace: define las carpetas donde se guardarán figuras, tablas y logs bajo models/fbcsp_lda/. Incluye utilidades para inicializar un logger limpio, guardar matrices de confusión (PNG + CSV) y anexar métricas a CSVs acumulativos.

In [8]:
# %% [PATHS & LOGGING] — rutas de salida + helpers para logs/figuras/tablas
import sys, logging, warnings
from datetime import datetime
from pathlib import Path
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.metrics import ConfusionMatrixDisplay
import mne

# Raíz del repo (este notebook está en models/fbcsp_lda/)
PROJ = Path('..').resolve().parent
DATA_PROC = PROJ / 'data' / 'processed'
OUT_ROOT = PROJ / 'models' / 'fbcsp_lda'
FIG_DIR  = OUT_ROOT / 'figures'
TAB_DIR  = OUT_ROOT / 'tables'
LOG_DIR  = OUT_ROOT / 'logs'
for d in (FIG_DIR, TAB_DIR, LOG_DIR):
    d.mkdir(parents=True, exist_ok=True)

print(f"Directorio de datos procesados: {DATA_PROC}")


Directorio de datos procesados: /root/Proyecto/EEG_Clasificador/data/processed


### Bloque 2 — FBCSP Helpers

Qué hace: helpers de modelo. Extraen X/y desde Epochs, aplican Filter-Bank + CSP por bandas y entrenan/escala LDA. Mantiene tus comentarios y añade compatibilidad con versiones de CSP.

In [9]:
# %% [HELPERS — comunes FBCSP/LOSO/Calibración/Logging]
# Reúne en un solo bloque:
#  - Descubrimiento de sujetos + DROP-only
#  - Perillas (_knobs_dict)
#  - FBCSP helpers (banco de filtros, picks motores, z-score por época, FBCSP transform)
#  - Clasificador (scaler + LDA)
#  - Split de calibración
#  - Logger

# ====== IMPORTS ======
import sys, logging, warnings, re
from glob import glob
from datetime import datetime
from pathlib import Path

import numpy as np
import pandas as pd
import mne

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from mne.decoding import CSP

# ====== RUTAS y ARCHIVOS AUXILIARES ======
# NOTA: este bloque asume que en un bloque anterior ya definiste:
#   PROJ, DATA_PROC, LOG_DIR  (del bloque PATHS & LOGGING original)
# Archivo opcional con sujetos a excluir (DROP-only)
STRICT_DROP_TXT = PROJ / 'reports' / 'tables' / '02_prepro' / 'subjects_strict_DROP.txt'
_re_sid = re.compile(r'^S\d{3}$')
# ====== FBCSP HELPERS ======
# Bancos de filtros
FB_BANDS_DENSE = [(f, f+2) for f in range(8, 30, 2)]                  # denso 8–30 por pasos de 2 Hz
FB_BANDS_CLASSIC = [(8,12), (12,16), (16,20), (20,24), (24,28), (28,30)]
DEFAULT_FB_BANDS = FB_BANDS_DENSE

# Nº de componentes CSP por sub-banda (típico 6–8 en bancos densos)
DEFAULT_N_CSP = 6

# LDA con shrinkage automático robusto
LDA_PARAMS = dict(solver='lsqr', shrinkage='auto')

# Picks motores (si deseas limitar; tus datos ya tienen 8 canales motores)
MOTOR_TOKENS = ['C3', 'CZ', 'C4', 'FC3', 'FC4', 'CP3', 'CPZ', 'CP4']

def _list_subject_fifs(fif_dir=DATA_PROC, pattern='S???_MI-epo.fif'):
    """Devuelve lista ordenada de rutas a los FIF de sujetos disponibles."""
    return sorted(glob(str(fif_dir / pattern)))

def _list_available_subjects(fif_dir=DATA_PROC):
    """IDs únicos SXXX disponibles en el directorio de FIF procesados."""
    files = _list_subject_fifs(fif_dir)
    return sorted({Path(f).stem.split('_')[0] for f in files})

def _read_drop_file(path: Path):
    """Lee archivo con IDs de sujetos a excluir (uno por línea, formato SXXX)."""
    if not path.exists():
        return set()
    s = set()
    with open(path, 'r', encoding='utf-8', errors='ignore') as f:
        for ln in f:
            sid = ln.strip().upper()
            if _re_sid.match(sid):
                s.add(sid)
    return s

def _strict_valid_from_drop(avail_ids):
    """
    Aplica DROP-only:
      - Lee STRICT_DROP_TXT si existe
      - Devuelve (lista válidos, info string)
    """
    drop = _read_drop_file(STRICT_DROP_TXT)
    avail = set(avail_ids)
    valid = sorted(avail - drop)
    info = f"DROP-only: {len(drop)} en DROP; válidos={len(valid)}/{len(avail)}"
    if not STRICT_DROP_TXT.exists():
        info += " (archivo DROP no encontrado → sin exclusiones)"
    return valid, info

# Helper para registrar “perillas” (config) en logs/CSV
def _knobs_dict(crop_window, motor_only, zscore_epoch, fb_bands, n_csp):
    return dict(
        crop_window=crop_window if crop_window is not None else None,
        motor_only=bool(motor_only),
        zscore_epoch=bool(zscore_epoch),
        fb_bands=str(fb_bands),
        n_csp=int(n_csp)
    )

def _epochs_to_Xy(epochs: mne.Epochs):
    """
    Extrae X e y (clases string) desde Epochs respetando event_id.
    X: (n_epochs, n_channels, n_times)
    y: (n_epochs,) etiquetas string según mapping event_id.
    """
    X = epochs.get_data()
    inv = {v: k for k, v in epochs.event_id.items()}  # int->clase
    y = np.array([inv[e[-1]] for e in epochs.events], dtype=object)
    return X, y

def _find_motor_chs(ch_names, tokens=MOTOR_TOKENS):
    """Devuelve índices de canales que contienen tokens motores (case-insensitive)."""
    up = [c.upper() for c in ch_names]
    picks = []
    for tok in tokens:
        TU = tok.upper()
        for i, name in enumerate(up):
            if TU in name:
                picks.append(i); break
    return sorted(set(picks))

def _epochwise_zscore(X, eps=1e-8):
    """
    Z-score por época y canal (normaliza a lo largo del tiempo).
    X: (n_epochs, n_channels, n_times) → mismo shape.
    """
    mean = X.mean(axis=-1, keepdims=True)
    std  = X.std(axis=-1, keepdims=True)
    return (X - mean) / (std + eps)

def _fit_fb_csp_transform(train_ep: mne.Epochs,
                          test_ep: mne.Epochs,
                          fb_bands=DEFAULT_FB_BANDS,
                          n_csp=DEFAULT_N_CSP,
                          motor_only=False,
                          zscore_epoch=False,
                          crop_window=None):
    """
    Aplica FBCSP con opciones:
      - crop_window=(tmin,tmax): recorta épocas
      - motor_only=True: usa solo canales motores comunes
      - zscore_epoch=True: z-score por época/canal antes de CSP
      - fb_bands: lista de sub-bandas [(fmin,fmax), ...]
    Devuelve (Xtr_fb, Xte_fb) con features concatenadas por sub-banda.
    """
    tr = train_ep.copy()
    te = test_ep.copy()

    if crop_window is not None:
        tmin, tmax = crop_window
        tr.crop(tmin, tmax)
        te.crop(tmin, tmax)

    if motor_only:
        picks = _find_motor_chs(tr.ch_names)
        if picks:
            tr.pick(picks)
            # Alinear canales del test con los del train
            te = te.copy().reorder_channels(tr.ch_names)

    Xtr_list, Xte_list = [], []
    y_tr = tr.events[:, -1]

    for (fmin, fmax) in fb_bands:
        tr_b = tr.copy().filter(fmin, fmax, picks='eeg', verbose=False)
        te_b = te.copy().filter(fmin, fmax, picks='eeg', verbose=False)

        Xtr = tr_b.get_data()
        Xte = te_b.get_data()

        if zscore_epoch:
            Xtr = _epochwise_zscore(Xtr)
            Xte = _epochwise_zscore(Xte)

        try:
            csp = CSP(n_components=n_csp, reg='ledoit_wolf', log=True, norm_trace=False)
        except TypeError:
            # Compatibilidad con MNE antiguos sin 'norm_trace'
            csp = CSP(n_components=n_csp, reg='ledoit_wolf', log=True)

        Xtr_c = csp.fit_transform(Xtr, y_tr)
        Xte_c = csp.transform(Xte)

        Xtr_list.append(Xtr_c)
        Xte_list.append(Xte_c)

    Xtr_fb = np.concatenate(Xtr_list, axis=1)
    Xte_fb = np.concatenate(Xte_list,  axis=1)
    return Xtr_fb, Xte_fb

def _fit_scale_lda(Xtr, ytr, Xte, lda_params=LDA_PARAMS):
    """
    Estandariza features (fit solo en train) y entrena LDA.
    Devuelve: (yhat, clf, scaler)
    """
    scaler = StandardScaler()
    Xtr_s = scaler.fit_transform(Xtr)
    Xte_s = scaler.transform(Xte)

    clf = LDA(**lda_params)
    clf.fit(Xtr_s, ytr)
    yhat = clf.predict(Xte_s)
    return yhat, clf, scaler

# ====== CALIBRACIÓN (para LOSO calibrado) ======
def _split_calibration(ep_te, k_per_class=5):
    """
    Toma k épocas por clase como calibración y el resto como evaluación.
    Si k_per_class <= 0 → (None, ep_te).
    """
    if k_per_class <= 0:
        return None, ep_te
    labels = ep_te.events[:, -1]
    ep_calib_list, ep_eval_list = [], []
    for code in np.unique(labels):
        idx = np.where(labels == code)[0]
        if len(idx) == 0:
            continue
        take = min(k_per_class, len(idx))
        sel = idx[:take]
        rem = idx[take:]
        if take > 0:
            ep_calib_list.append(ep_te.copy()[sel])
        if len(rem) > 0:
            ep_eval_list.append(ep_te.copy()[rem])
    ep_calib = mne.concatenate_epochs(ep_calib_list, on_mismatch='ignore') if ep_calib_list else None
    ep_eval  = mne.concatenate_epochs(ep_eval_list,  on_mismatch='ignore') if ep_eval_list  else ep_te
    return ep_calib, ep_eval

# ====== LOGGER ======
def _init_logger(run_name: str):
    """
    Crea un logger que escribe a consola y a TXT en models/fbcsp_lda/logs/.
    Reduce la verbosidad de MNE para que los logs sean legibles.
    """
    ts = datetime.now().strftime("%Y%m%d-%H%M%S")
    log_path = LOG_DIR / f"{ts}_{run_name}.txt"

    logger = logging.getLogger(run_name)
    logger.setLevel(logging.INFO)
    logger.handlers.clear()

    fmt = logging.Formatter("[%(asctime)s] %(levelname)s: %(message)s", datefmt="%H:%M:%S")
    ch = logging.StreamHandler(stream=sys.stdout); ch.setLevel(logging.INFO); ch.setFormatter(fmt)
    fh = logging.FileHandler(log_path, encoding="utf-8"); fh.setLevel(logging.INFO); fh.setFormatter(fmt)
    logger.addHandler(ch); logger.addHandler(fh)

    # Silenciar ruido externo (sin afectar tus prints/logs)
    mne.set_log_level("ERROR")
    warnings.filterwarnings("ignore", category=UserWarning, module="mne")
    warnings.filterwarnings("ignore", category=RuntimeWarning, module="mne")
    return logger, log_path


### Bloque 3 — Inspección de datos

Qué hace: muestra rutas y un listado rápido del contenido de data/ y data/processed/ para verificar que los FIF están donde esperamos

In [10]:
# %% [Inspect data folders]
# Celda añadida automáticamente: muestra rutas y lista contenidos de data y data/processed
try:
    print(f"PROJ: {PROJ}")
    print(f"DATA: {PROJ / 'data'}")
    print(f"DATA_PROC: {DATA_PROC}")
    print('\nContenido de data (top-level):')
    data_dir = PROJ / 'data'
    if data_dir.exists():
        for p in sorted(data_dir.iterdir()):
            print(f" - {p.name}{'/' if p.is_dir() else ''}")
    else:
        print('  (no existe)')

    print('\nContenido de data/processed (muestras):')
    if DATA_PROC.exists():
        for p in sorted(DATA_PROC.glob('*'))[:50]:
            print(f" - {p.name}")
    else:
        print('  (no existe)')
except Exception as e:
    print('Error inspeccionando data:', e)


PROJ: /root/Proyecto/EEG_Clasificador
DATA: /root/Proyecto/EEG_Clasificador/data
DATA_PROC: /root/Proyecto/EEG_Clasificador/data/processed

Contenido de data (top-level):
 - cache/
 - processed/
 - raw/

Contenido de data/processed (muestras):
 - S001_MI-epo.fif
 - S002_MI-epo.fif
 - S003_MI-epo.fif
 - S004_MI-epo.fif
 - S005_MI-epo.fif
 - S006_MI-epo.fif
 - S007_MI-epo.fif
 - S008_MI-epo.fif
 - S009_MI-epo.fif
 - S010_MI-epo.fif
 - S011_MI-epo.fif
 - S012_MI-epo.fif
 - S013_MI-epo.fif
 - S014_MI-epo.fif
 - S015_MI-epo.fif
 - S016_MI-epo.fif
 - S017_MI-epo.fif
 - S018_MI-epo.fif
 - S019_MI-epo.fif
 - S020_MI-epo.fif
 - S021_MI-epo.fif
 - S022_MI-epo.fif
 - S023_MI-epo.fif
 - S024_MI-epo.fif
 - S025_MI-epo.fif
 - S026_MI-epo.fif
 - S027_MI-epo.fif
 - S028_MI-epo.fif
 - S029_MI-epo.fif
 - S030_MI-epo.fif
 - S031_MI-epo.fif
 - S032_MI-epo.fif
 - S033_MI-epo.fif
 - S034_MI-epo.fif
 - S035_MI-epo.fif
 - S036_MI-epo.fif
 - S037_MI-epo.fif
 - S039_MI-epo.fif
 - S040_MI-epo.fif
 - S041_MI-epo.

### Bloque 4 — Intra-sujeto (k-Fold CV) con logs + guardado

Qué hace: ejecuta CV por sujeto con FBCSP+LDA, imprime métricas limpias, guarda matriz de confusión (PNG/CSV), y escribe métricas en tables/metrics_intra.csv. Además genera un TXT en logs/ con el detalle de la corrida.

Añade argumentos crop_window, motor_only, zscore_epoch, fb_bands, n_csp. Guarda lo de siempre (matriz, métricas, log) pero ahora puedes probar rápidamente ventanas/picks

In [11]:
# %% [INTRA — todos los sujetos, con timestamp y artefactos consolidados + fila GLOBAL]
from glob import glob
from pathlib import Path
from math import ceil
from datetime import datetime
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import mne

def _discover_subject_ids(fif_dir=DATA_PROC, pattern='S???_MI-epo.fif'):
    files = sorted(glob(str(fif_dir / pattern)))
    return [Path(f).stem.split('_')[0] for f in files]


def run_intra_all(
    fif_dir=DATA_PROC,
    k=5,
    random_state=42,
    crop_window=(0.5, 3.5),
    motor_only=True,
    zscore_epoch=True,
    fb_bands=DEFAULT_FB_BANDS,
    n_csp=DEFAULT_N_CSP,
    max_subplots_per_fig=12,
    n_cols=4,
    save_txt_name=None,            # opcional: nombre base TXT (se antepone timestamp)
    save_csv_name=None             # opcional: nombre base CSV (se antepone timestamp)
):
    """
    Ejecuta INTRA (k-fold) en TODOS los sujetos.
    Guarda: 1 log con timestamp, 1 CSV y 1 TXT (ambos con fila GLOBAL),
    y mosaicos de matrices de confusión por sujeto (sin archivos por sujeto).
    """
    ts = datetime.now().strftime("%Y%m%d-%H%M%S")
    run_tag = f"intra_all_{ts}"

    # Log global
    logger, log_path = _init_logger(run_name=run_tag)
    logger.info(f"[RUN {run_tag}] Inicio de ejecución INTRA")
    logger.info(f"Parámetros: k={k}, crop_window={crop_window}, motor_only={motor_only}, "
                f"zscore_epoch={zscore_epoch}, n_csp={n_csp}, fb_bands={len(fb_bands)}")

    subject_ids = _discover_subject_ids(fif_dir)
    if not subject_ids:
        print("No se encontraron sujetos en", fif_dir)
        return None

    logger.info(f"Sujetos detectados: {subject_ids}")
    print(f"[INTRA ALL] sujetos detectados: {subject_ids}")

    rows_summary = []
    cm_items = []

    for subject_id in subject_ids:
        fif_path = fif_dir / f"{subject_id}_MI-epo.fif"
        epochs = mne.read_epochs(fif_path, preload=True, verbose=False)

        _, y_str = _epochs_to_Xy(epochs)
        le = LabelEncoder(); y = le.fit_transform(y_str)
        classes = list(le.classes_)

        logger.info(f"== {subject_id} | n_epochs={len(y)} | clases={classes} | sfreq={epochs.info['sfreq']}")
        skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=random_state)
        accs, f1s = [], []
        cm_sum = np.zeros((len(classes), len(classes)), dtype=int)

        for fold, (tr_idx, te_idx) in enumerate(skf.split(np.zeros(len(y)), y), start=1):
            ep_tr = epochs[tr_idx]; ep_te = epochs[te_idx]
            with mne.utils.use_log_level("ERROR"):
                Xtr_fb, Xte_fb = _fit_fb_csp_transform(
                    ep_tr, ep_te,
                    fb_bands=fb_bands,
                    n_csp=n_csp,
                    motor_only=motor_only,
                    zscore_epoch=zscore_epoch,
                    crop_window=crop_window
                )
            yhat, clf, scaler = _fit_scale_lda(Xtr_fb, y[tr_idx], Xte_fb)

            acc = accuracy_score(y[te_idx], yhat)
            f1m = f1_score(y[te_idx], yhat, average='macro')
            cm_sum += confusion_matrix(y[te_idx], yhat, labels=np.arange(len(classes)))
            accs.append(acc); f1s.append(f1m)
            logger.info(f"[{subject_id} | fold {fold}] acc={acc:.3f} | f1m={f1m:.3f}")

        acc_mu, acc_sd = float(np.mean(accs)), float(np.std(accs))
        f1_mu,  f1_sd  = float(np.mean(f1s)),  float(np.std(f1s))

        logger.info(f"[{subject_id}] ACC={acc_mu:.3f}±{acc_sd:.3f} | F1m={f1_mu:.3f}±{f1_sd:.3f}")
        rows_summary.append(dict(
            subject=subject_id,
            acc_mean=acc_mu,
            f1_macro_mean=f1_mu,
            k=k,
            n_classes=len(classes),
            crop=str(crop_window),
            motor_only=bool(motor_only),
            zscore_epoch=bool(zscore_epoch),
            n_csp=int(n_csp),
            fb_bands=len(fb_bands)
        ))
        cm_items.append((subject_id, cm_sum, classes))

    # --- CSV/TXT (con fila GLOBAL)
    df = pd.DataFrame(rows_summary).sort_values("subject")

    acc_mu = float(df['acc_mean'].mean()) if not df.empty else 0.0
    acc_sd = float(df['acc_mean'].std(ddof=0)) if not df.empty else 0.0
    f1_mu  = float(df['f1_macro_mean'].mean()) if not df.empty else 0.0
    f1_sd  = float(df['f1_macro_mean'].std(ddof=0)) if not df.empty else 0.0

    df_global = pd.DataFrame([{
        'subject': 'GLOBAL',
        'acc_mean': acc_mu,
        'f1_macro_mean': f1_mu,
        'k': k,
        'n_classes': int(df['n_classes'].mean()) if 'n_classes' in df.columns and not df.empty else 0,
        'crop': str(crop_window),
        'motor_only': bool(motor_only),
        'zscore_epoch': bool(zscore_epoch),
        'n_csp': int(n_csp),
        'fb_bands': len(fb_bands)
    }])
    df_out = pd.concat([df, df_global], ignore_index=True)

    out_csv = (TAB_DIR / f"{ts}_{save_csv_name}") if save_csv_name else (TAB_DIR / f"metrics_intra_all_{ts}.csv")
    df_out.to_csv(out_csv, index=False)
    logger.info(f"Resumen CSV guardado → {out_csv}")
    print("Resumen INTRA (todos) →", out_csv)

    logger.info(f"[GLOBAL INTRA] ACC={acc_mu:.3f}±{acc_sd:.3f} | F1m={f1_mu:.3f}±{f1_sd:.3f}")
    print(f"[GLOBAL INTRA] ACC={acc_mu:.3f}±{acc_sd:.3f} | F1m={f1_mu:.3f}±{f1_sd:.3f}")

    try:
        display(df_out)
    except Exception:
        pass

    out_txt = (LOG_DIR / f"{ts}_{save_txt_name}") if save_txt_name else (LOG_DIR / f"metrics_intra_all_{ts}.txt")
    with open(out_txt, "w", encoding="utf-8") as f:
        f.write(f"INTRA-SUJETO (k-fold) — Métricas por sujeto (incluye GLOBAL)\n")
        f.write(f"Generado: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        f.write(f"Total filas: {len(df_out)}\n\n")
        header = df_out.columns.tolist()
        f.write(" | ".join(header) + "\n")
        f.write("-" * 90 + "\n")
        for _, row in df_out.iterrows():
            vals = []
            for kcol in header:
                v = row[kcol]
                vals.append(f"{v:.4f}" if isinstance(v, float) else str(int(v)) if isinstance(v, (np.integer,)) else str(v))
            f.write(" | ".join(vals) + "\n")
    logger.info(f"Métricas TXT guardadas → {out_txt}")
    print("Métricas TXT guardadas →", out_txt)

    # --- Mosaicos de matrices de confusión
    if cm_items:
        n = len(cm_items)
        per_fig = max(1, int(max_subplots_per_fig))
        n_figs = ceil(n / per_fig)
        n_rows_per_fig = lambda count: ceil(count / n_cols)

        for fig_idx in range(n_figs):
            start = fig_idx * per_fig
            end   = min((fig_idx + 1) * per_fig, n)
            chunk = cm_items[start:end]
            count = len(chunk)
            n_rows = n_rows_per_fig(count)

            fig, axes = plt.subplots(n_rows, n_cols, figsize=(4.5*n_cols, 3.8*n_rows), dpi=140)
            axes = np.atleast_2d(axes).flatten()

            for ax_i, (sid, cm_sum, classes) in enumerate(chunk):
                ax = axes[ax_i]
                disp = ConfusionMatrixDisplay(cm_sum, display_labels=classes)
                disp.plot(ax=ax, cmap="Blues", colorbar=False, values_format='d')
                ax.set_title(f"{sid}")
                ax.set_xlabel(""); ax.set_ylabel("")
            for j in range(ax_i + 1, len(axes)):
                axes[j].axis("off")

            out_png = FIG_DIR / f"intra_all_confusions_{ts}_p{fig_idx+1}.png"
            fig.suptitle(f"Intra — Matrices de confusión (página {fig_idx+1}/{n_figs})", y=0.995, fontsize=14)
            fig.tight_layout(rect=[0, 0, 1, 0.97])
            fig.savefig(out_png)
            plt.close(fig)
            logger.info(f"Figura consolidada → {out_png}")
            print("Figura consolidada →", out_png)

    logger.info(f"Log global de esta corrida → {log_path}")
    print(f"Log global → {log_path}")

    return df_out


### Bloque 5 — Cross-sujeto (LOSO) con logs + guardado

Qué hace: para cada sujeto como test, entrena en el resto, calcula métricas y guarda una matriz de confusión por sujeto y una global. También guarda métricas por sujeto en tables/metrics_loso_per_subject.csv y un resumen global en tables/metrics_loso.csv, además de un TXT en logs/

- run_loso(..., use_strict=True) hace LOSO clásico sobre el conjunto resuelto (con Strict si está ON).

- run_loso_single(test_subject, ...) entrena con todos los demás y prueba sólo en ese sujeto (Strict opcional).

- Incluye utilidades de selección Strict y reemplazo para subject_list.

In [12]:
# %% [INTER-SUBJECT CV desde JSON de folds — NO usa archivo DROP; usa todos los sujetos del JSON]
import json
import numpy as np
import pandas as pd
import mne
from math import ceil
from datetime import datetime
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    confusion_matrix,
    ConfusionMatrixDisplay,
    classification_report,   # <<< NUEVO
)
import matplotlib.pyplot as plt
from pathlib import Path

def run_inter_subject_cv_from_json(
    fif_dir=DATA_PROC,
    folds_json_path=None,
    crop_window=(0.5, 3.5),
    motor_only=True,
    zscore_epoch=True,
    fb_bands=DEFAULT_FB_BANDS,
    n_csp=DEFAULT_N_CSP,
    calibrate_n=None,                  # calibración opcional con sujetos de test
    val_ratio_subjects=0.16,           # % sujetos de TRAIN que van a VALID (≈ 13/82 ≈ 0.16)
    random_state=42,                   # para reproducibilidad del split por sujeto
    max_subplots_per_fig=12,
    n_cols=4,
    save_csv_name=None,
    save_txt_name=None
):
    """
    Inter-subject CV usando folds del JSON, con VALIDACIÓN INTERNA POR SUJETOS.
    - El split de validación se hace a nivel SUJETO dentro del conjunto de train de cada fold.
    - FBCSP/Scaler/LDA se ajustan SOLO con sujetos de train (sin val ni test).
    - Se reportan métricas en valid y en test.
    - Classification report (TEST) por fold impreso y guardado en TXT.
    """
    ts = datetime.now().strftime("%Y%m%d-%H%M%S")
    run_tag = f"inter_subject_cv_json_{ts}"
    logger, log_path = _init_logger(run_name=run_tag)
    knobs = _knobs_dict(crop_window, motor_only, zscore_epoch, fb_bands, n_csp)
    logger.info(f"[RUN {run_tag}] Inter-Subject CV (con VALID interno por sujetos)")
    logger.info(f"Perillas: {knobs} | val_ratio_subjects={val_ratio_subjects:.2f} | calibrate_n={calibrate_n}")

    # JSON de folds
    if folds_json_path is None:
        folds_json_path = PROJ / 'models' / 'folds' / 'Kfold5.json'
    folds_json_path = Path(folds_json_path)
    if not folds_json_path.exists():
        raise FileNotFoundError(f"No se encontró folds JSON en {folds_json_path}")

    with open(folds_json_path, "r", encoding="utf-8") as f:
        payload = json.load(f)

    folds = payload.get("folds", [])
    subject_ids_json = payload.get("subject_ids", [])
    logger.info(f"Folds cargadas: {len(folds)} | sujetos en JSON: {len(subject_ids_json)}")

    # Cargar epochs por sujeto
    ep_map = {}
    for sid in subject_ids_json:
        fif_path = Path(fif_dir) / f"{sid}_MI-epo.fif"
        if fif_path.exists():
            try:
                ep_map[sid] = mne.read_epochs(str(fif_path), preload=True, verbose=False)
            except Exception as e:
                logger.warning(f"Error leyendo {fif_path} para {sid}: {e}")
        else:
            logger.warning(f"Falta archivo FIF para {sid}: {fif_path}")

    # Resultados
    rows = []
    cm_items = []
    cm_global = None
    classes_global = None
    per_fold_reports = []  # <<< NUEVO: guardamos classification_report (TEST) por fold

    for f in folds:
        fold_i = int(f.get("fold"))
        train_sids = [sid for sid in f.get("train", []) if sid in ep_map]
        test_sids  = [sid for sid in f.get("test", [])  if sid in ep_map]

        logger.info(f"[Fold {fold_i}] train({len(train_sids)}): {train_sids}")
        logger.info(f"[Fold {fold_i}] test ({len(test_sids)}): {test_sids}")

        if len(train_sids) == 0 or len(test_sids) == 0:
            logger.warning(f"[Fold {fold_i}] faltan sujetos train/test — saltando fold.")
            continue

        # ---------- VALIDACIÓN INTERNA POR SUJETOS ----------
        rng = np.random.RandomState(random_state + fold_i)
        n_val_subj = max(1, int(round(len(train_sids) * float(val_ratio_subjects))))
        val_indices = rng.choice(len(train_sids), size=n_val_subj, replace=False)
        val_sids = sorted([train_sids[i] for i in val_indices])
        tr_sids  = sorted([sid for sid in train_sids if sid not in set(val_sids)])

        logger.info(f"[Fold {fold_i}] split interno → train_sids={len(tr_sids)}, val_sids={len(val_sids)}")

        # Concatenar epochs por split
        ep_tr  = mne.concatenate_epochs([ep_map[sid] for sid in tr_sids],  on_mismatch='ignore')
        ep_val = mne.concatenate_epochs([ep_map[sid] for sid in val_sids], on_mismatch='ignore')
        ep_te  = mne.concatenate_epochs([ep_map[sid] for sid in test_sids], on_mismatch='ignore')

        # Alinear canales
        try:
            ep_val = ep_val.copy().reorder_channels(ep_tr.ch_names)
            ep_te  = ep_te.copy().reorder_channels(ep_tr.ch_names)
        except Exception as e:
            logger.warning(f"[Fold {fold_i}] reorder_channels: {e}")

        # Etiquetas
        _, y_tr_str  = _epochs_to_Xy(ep_tr)
        _, y_val_str = _epochs_to_Xy(ep_val)
        _, y_te_str  = _epochs_to_Xy(ep_te)

        le = LabelEncoder().fit(np.concatenate([y_tr_str, y_val_str, y_te_str]))
        y_tr  = le.transform(y_tr_str)
        y_val = le.transform(y_val_str)
        y_te  = le.transform(y_te_str)
        classes = list(le.classes_)

        if classes_global is None:
            classes_global = classes
            cm_global = np.zeros((len(classes), len(classes)), dtype=int)

        # ---------- FEATURES FBCSP (ajuste SOLO con TRAIN-SUBJECTS) ----------
        with mne.utils.use_log_level("ERROR"):
            # Fit en ep_tr, transform en ep_val y ep_te
            Xtr_fb, Xval_fb = _fit_fb_csp_transform(
                ep_tr, ep_val,
                fb_bands=fb_bands,
                n_csp=n_csp,
                motor_only=motor_only,
                zscore_epoch=zscore_epoch,
                crop_window=crop_window
            )
            # Obtener Xte con el mismo ajuste (re-aplicando con ep_tr como "ajuste")
            _, Xte_fb = _fit_fb_csp_transform(
                ep_tr, ep_te,
                fb_bands=fb_bands,
                n_csp=n_csp,
                motor_only=motor_only,
                zscore_epoch=zscore_epoch,
                crop_window=crop_window
            )

        # ---------- ENTRENAR CLASIFICADOR SOLO CON TRAIN ----------
        # Ajuste scaler/LDA con TRAIN y evaluar en VAL y TEST
        yhat_val, clf_val, scaler_val = _fit_scale_lda(Xtr_fb, y_tr, Xval_fb)
        acc_val = accuracy_score(y_val, yhat_val)
        f1m_val = f1_score(y_val, yhat_val, average='macro')
        logger.info(f"[Fold {fold_i}] VAL   acc={acc_val:.4f} | f1m={f1m_val:.4f} | n_val={len(y_val)}")

        # --------- TEST normal o con calibración opcional ----------
        if calibrate_n is None or calibrate_n <= 0:
            # usar el mismo clf/scaler ajustado con TRAIN para predecir TEST
            yhat_te = clf_val.predict(scaler_val.transform(Xte_fb))
        else:
            # calibración con n sujetos de TEST (no usa VAL)
            n_subjs = min(int(calibrate_n), len(test_sids))
            calib_sids = test_sids[:n_subjs]
            rest_sids  = test_sids[n_subjs:]
            ep_calib = mne.concatenate_epochs([ep_map[sid] for sid in calib_sids], on_mismatch='ignore')
            ep_te_rest = mne.concatenate_epochs([ep_map[sid] for sid in rest_sids], on_mismatch='ignore') if rest_sids else ep_calib

            # Reordenar y etiquetas
            try:
                ep_calib   = ep_calib.copy().reorder_channels(ep_tr.ch_names)
                ep_te_rest = ep_te_rest.copy().reorder_channels(ep_tr.ch_names)
            except Exception as e:
                logger.warning(f"[Fold {fold_i}] reorder (calib/test_rest): {e}")

            _, y_calib_str = _epochs_to_Xy(ep_calib)
            y_calib = le.transform(y_calib_str)

            # Recalcular features con TRAIN+CALIB (ajuste) y transformar TEST-REST
            with mne.utils.use_log_level("ERROR"):
                ep_train_plus_calib = mne.concatenate_epochs([ep_tr, ep_calib], on_mismatch='ignore')
                Xtr_comb, Xte_rest = _fit_fb_csp_transform(
                    ep_train_plus_calib, ep_te_rest,
                    fb_bands=fb_bands,
                    n_csp=n_csp,
                    motor_only=motor_only,
                    zscore_epoch=zscore_epoch,
                    crop_window=crop_window
                )

            y_tr_comb = np.concatenate([y_tr, y_calib])
            yhat_te = _fit_scale_lda(Xtr_comb, y_tr_comb, Xte_rest)[0]
            if rest_sids:
                _, y_te_rest_str = _epochs_to_Xy(ep_te_rest)
                y_te = le.transform(y_te_rest_str)
            else:
                y_te = y_tr_comb  # caso extremo: calibras todo el test

        # ---------- MÉTRICAS TEST ----------
        acc = accuracy_score(y_te, yhat_te)
        f1m = f1_score(y_te, yhat_te, average='macro')
        cm = confusion_matrix(y_te, yhat_te, labels=np.arange(len(classes)))
        cm_global += cm

        # <<< NUEVO: Classification report (TEST) por fold
        cls_rep = classification_report(y_te, yhat_te, target_names=classes, digits=4)
        logger.info(f"[Fold {fold_i}] Classification report (TEST):\n{cls_rep}")
        print(f"[Fold {fold_i}] Classification report (TEST):\n{cls_rep}")
        per_fold_reports.append((fold_i, cls_rep))

        logger.info(f"[Fold {fold_i}] TEST  acc={acc:.4f} | f1m={f1m:.4f} | n_test={len(y_te)}")
        rows.append(dict(
            fold=int(fold_i),
            train_subjects=",".join(tr_sids),
            val_subjects=",".join(val_sids),
            test_subjects=",".join(test_sids),
            val_acc=float(acc_val),
            val_f1_macro=float(f1m_val),
            acc=float(acc),
            f1_macro=float(f1m),
            n_val=int(len(y_val)),
            n_test=int(len(y_te))
        ))
        cm_items.append((f"fold_{fold_i}", cm, classes))

    # ---------- Consolidados ----------
    df_rows = pd.DataFrame(rows).sort_values("fold") if rows else pd.DataFrame()
    acc_mu   = float(df_rows['acc'].mean()) if not df_rows.empty else 0.0
    f1_mu    = float(df_rows['f1_macro'].mean()) if not df_rows.empty else 0.0
    val_mu   = float(df_rows['val_acc'].mean()) if not df_rows.empty else 0.0
    valf1_mu = float(df_rows['val_f1_macro'].mean()) if not df_rows.empty else 0.0

    if not df_rows.empty:
        df_rows = pd.concat([df_rows, pd.DataFrame([{
            'fold': 0,
            'train_subjects': 'GLOBAL',
            'val_subjects': 'GLOBAL',
            'test_subjects': 'GLOBAL',
            'val_acc': val_mu,
            'val_f1_macro': valf1_mu,
            'acc': acc_mu,
            'f1_macro': f1_mu,
            'n_val': int(df_rows['n_val'].sum()),
            'n_test': int(df_rows['n_test'].sum())
        }])], ignore_index=True)

    out_csv = (TAB_DIR / f"{ts}_{save_csv_name}") if save_csv_name else (TAB_DIR / f"metrics_inter_subject_cv_{ts}.csv")
    df_rows.to_csv(out_csv, index=False)
    logger.info(f"CSV consolidado → {out_csv}")
    print("CSV consolidado →", out_csv)

    out_txt = (LOG_DIR / f"{ts}_{save_txt_name}") if save_txt_name else (LOG_DIR / f"metrics_inter_subject_cv_{ts}.txt")
    with open(out_txt, "w", encoding="utf-8") as f:
        f.write("INTER-SUBJECT CV (folds JSON) — Con VALID interno por sujetos\n")
        f.write(f"Generado: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        f.write(f"Total filas: {len(df_rows)}\n\n")
        if not df_rows.empty:
            header = df_rows.columns.tolist()
            f.write(" | ".join(header) + "\n")
            f.write("-" * 140 + "\n")
            for _, row in df_rows.iterrows():
                vals = []
                for kcol in header:
                    v = row[kcol]
                    if isinstance(v, float):
                        vals.append(f"{v:.4f}")
                    elif isinstance(v, (np.integer,)):
                        vals.append(str(int(v)))
                    else:
                        vals.append(str(v))
                f.write(" | ".join(vals) + "\n")
    logger.info(f"TXT consolidado → {out_txt}")
    print("TXT consolidado →", out_txt)

    # <<< NUEVO: TXT con todos los classification reports por fold (TEST)
    reports_txt = LOG_DIR / f"classification_reports_by_fold_{ts}.txt"
    with open(reports_txt, "w", encoding="utf-8") as f:
        f.write("INTER-SUBJECT CV — Classification reports por fold (TEST)\n")
        f.write(f"Generado: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
        for fold_i, rep in sorted(per_fold_reports, key=lambda x: x[0]):
            f.write(f"[Fold {fold_i}] Classification report (TEST)\n")
            f.write(rep)
            if not rep.endswith("\n"):
                f.write("\n")
            f.write("-" * 80 + "\n")
    logger.info(f"TXT de classification reports → {reports_txt}")
    print("TXT de classification reports →", reports_txt)

    # Mosaicos de confusión por fold
    if cm_items:
        n = len(cm_items)
        per_fig = max(1, int(max_subplots_per_fig))
        n_figs = ceil(n / per_fig)
        n_rows_per_fig = lambda count: ceil(count / n_cols)
        for fig_idx in range(n_figs):
            start = fig_idx * per_fig
            end   = min((fig_idx + 1) * per_fig, n)
            chunk = cm_items[start:end]
            count = len(chunk)
            n_rows = n_rows_per_fig(count)

            fig, axes = plt.subplots(n_rows, n_cols, figsize=(4.5*n_cols, 3.8*n_rows), dpi=140)
            axes = np.atleast_2d(axes).flatten()
            for ax_i, (label, cm_sum, classes) in enumerate(chunk):
                ax = axes[ax_i]
                disp = ConfusionMatrixDisplay(cm_sum, display_labels=classes)
                disp.plot(ax=ax, cmap="Blues", colorbar=False, values_format='d')
                ax.set_title(f"{label}")
                ax.set_xlabel(""); ax.set_ylabel("")
            for j in range(ax_i + 1, len(axes)):
                axes[j].axis("off")

            out_png = FIG_DIR / f"inter_subject_confusions_{ts}_p{fig_idx+1}.png"
            fig.suptitle(f"Inter-Subject CV — Matrices de confusión (página {fig_idx+1}/{n_figs})", y=0.995, fontsize=14)
            fig.tight_layout(rect=[0, 0, 1, 0.97])
            fig.savefig(out_png)
            plt.close(fig)
            logger.info(f"Figura consolidada → {out_png}")
            print("Figura consolidada →", out_png)

    # Matriz GLOBAL
    if cm_global is not None and classes_global is not None:
        fig, ax = plt.subplots(figsize=(6.5, 5.2), dpi=140)
        disp = ConfusionMatrixDisplay(cm_global, display_labels=classes_global)
        disp.plot(ax=ax, cmap="Blues", colorbar=True, values_format='d')
        ax.set_title("Inter-Subject CV — Matriz de confusión GLOBAL (sumatoria folds)")
        fig.tight_layout()
        out_png_glob = FIG_DIR / f"inter_subject_global_confusion_{ts}.png"
        fig.savefig(out_png_glob)
        plt.close(fig)
        logger.info(f"Matriz GLOBAL → {out_png_glob}")
        print("Matriz GLOBAL →", out_png_glob)

    logger.info(f"[GLOBAL] VAL_acc={val_mu:.3f} | VAL_f1m={valf1_mu:.3f} | TEST_acc={acc_mu:.3f} | TEST_f1m={f1_mu:.3f}")
    print(f"[GLOBAL] VAL_acc={val_mu:.3f} | VAL_f1m={valf1_mu:.3f} | TEST_acc={acc_mu:.3f} | TEST_f1m={f1_mu:.3f}")
    logger.info(f"Log global de esta corrida → {log_path}")
    print(f"Log global → {log_path}")

    return df_rows.reset_index(drop=True)


### Bloque 7 — Ejemplos de ejecución

Qué hace: muestra cómo lanzar los “batch” por defecto (4 intra, 2 LOSO) y cómo pasar propia lista de sujetos.

In [13]:
# INTRA en todos los sujetos
# df_intra_optimal = run_intra_all(
#     k=5,
#     random_state=42,
#     crop_window=(0.5, 4.5),      # VENTANA ORIGINAL PROBADA
#     motor_only=True,
#     zscore_epoch=False,
#     fb_bands=FB_BANDS_DENSE,     # 11 BANDS ORIGINALES
#     n_csp=4,
#     save_txt_name="intra_optimal_proven.txt"
# )

# Inter-subject CV usando JSON de folds (reemplaza LOSO clásico)
# Ajusta folds_json_path si usaste otra ruta; por defecto busca PROJ/models/folds/Kfold5.json
df_inter = run_inter_subject_cv_from_json(
    fif_dir=DATA_PROC,
    folds_json_path=PROJ / 'models' / 'folds' / 'Kfold5.json',
    crop_window=(0.5, 3.5),
    motor_only=True,
    zscore_epoch=False,
    fb_bands=FB_BANDS_DENSE,
    n_csp=4,
    calibrate_n=5,
    save_csv_name="inter_subject_cv_from_json.csv"
)


[06:24:52] INFO: [RUN inter_subject_cv_json_20251013-062452] Inter-Subject CV (con VALID interno por sujetos)


INFO:inter_subject_cv_json_20251013-062452:[RUN inter_subject_cv_json_20251013-062452] Inter-Subject CV (con VALID interno por sujetos)


[06:24:52] INFO: Perillas: {'crop_window': (0.5, 3.5), 'motor_only': True, 'zscore_epoch': False, 'fb_bands': '[(8, 10), (10, 12), (12, 14), (14, 16), (16, 18), (18, 20), (20, 22), (22, 24), (24, 26), (26, 28), (28, 30)]', 'n_csp': 4} | val_ratio_subjects=0.16 | calibrate_n=5


INFO:inter_subject_cv_json_20251013-062452:Perillas: {'crop_window': (0.5, 3.5), 'motor_only': True, 'zscore_epoch': False, 'fb_bands': '[(8, 10), (10, 12), (12, 14), (14, 16), (16, 18), (18, 20), (20, 22), (22, 24), (24, 26), (26, 28), (28, 30)]', 'n_csp': 4} | val_ratio_subjects=0.16 | calibrate_n=5


[06:24:52] INFO: Folds cargadas: 5 | sujetos en JSON: 103


...

Matriz GLOBAL → /root/Proyecto/EEG_Clasificador/models/fbcsp_lda/figures/inter_subject_global_confusion_20251013-062452.png
[06:41:56] INFO: [GLOBAL] VAL_acc=0.355 | VAL_f1m=0.354 | TEST_acc=0.375 | TEST_f1m=0.372


INFO:inter_subject_cv_json_20251013-062452:[GLOBAL] VAL_acc=0.355 | VAL_f1m=0.354 | TEST_acc=0.375 | TEST_f1m=0.372


[GLOBAL] VAL_acc=0.355 | VAL_f1m=0.354 | TEST_acc=0.375 | TEST_f1m=0.372
[06:41:56] INFO: Log global de esta corrida → /root/Proyecto/EEG_Clasificador/models/fbcsp_lda/logs/20251013-062452_inter_subject_cv_json_20251013-062452.txt


INFO:inter_subject_cv_json_20251013-062452:Log global de esta corrida → /root/Proyecto/EEG_Clasificador/models/fbcsp_lda/logs/20251013-062452_inter_subject_cv_json_20251013-062452.txt


Log global → /root/Proyecto/EEG_Clasificador/models/fbcsp_lda/logs/20251013-062452_inter_subject_cv_json_20251013-062452.txt
